# Wine Quality Preprocessing (Oversampling)

Made by: Anthony Kevin Oktavius

Library Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import klib
import plotly
import plotly.express as px
import shap
import hyperopt
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from scipy.stats import zscore
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In this section, I'm going to look at Wine Quality Oversampling Technique. In our case, we will use 3 types of oversampling technique, which are:

* Random Over Sampling
* SMOTE
* ADASYN

## Random Over Sampling

![Random Oversampling](https://miro.medium.com/max/375/1*aKJJOozIlVVH1gT-4rYy4w.png)

Image Source: https://miro.medium.com/max/375/1*aKJJOozIlVVH1gT-4rYy4w.png

![Random Oversampling Plot](https://imbalanced-learn.org/stable/_images/sphx_glr_plot_comparison_over_sampling_002.png)

Image Source: https://imbalanced-learn.org/stable/_images/sphx_glr_plot_comparison_over_sampling_002.png

Random Oversampling includes duplicating examples from minority classes. The duplicated minority examples will then be added to the original dataset, so it allows the data to be selected again.

This technique is effective when we'd like to normalize skewed distributions (mean and standard deviation are not 0). However, in some cases, doing this technique can cause the algorithm to overfit the minority class, since the same data row can be duplicated again.

## SMOTE

![SMOTE](https://miro.medium.com/max/1400/1*CG712DHQL_ZMS3gAEGd_5w.jpeg)

Image Source: https://miro.medium.com/max/1400/1*CG712DHQL_ZMS3gAEGd_5w.jpeg

Unlike Random Over Sampling where we copy the values from the actual dataset, SMOTE creates new synthetic observations/ new data points.

Here are the summary steps on SMOTE Process:
* Plot the data points.
* Identify feature vector and its nearest neighbour
* Take the linear distance difference between the two
* Multiply the difference with a random number between 0 and 1
* Identify a new point on the line segment by adding the random number to feature vector
* Repeat the process for identified feature vectors, until our defined data points that we’d like to create.

## ADASYN

In [ ]:
## SMOTE